<a href="https://colab.research.google.com/github/vaasu14/pyspark/blob/main/starting_with_Pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
!pwd

/content


In [ ]:
!wget -q https://www-us.apache.org/dist/spark/spark-3.1.1//spark-3.1.1-bin-hadoop3.2.tgz  

In [ ]:
!tar xf spark-3.1.1-bin-hadoop3.2.tgz

In [ ]:
!pwd

/content


In [ ]:
cd /content/spark-3.1.1-bin-hadoop3.2

/content/spark-3.1.1-bin-hadoop3.2


In [ ]:
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [ ]:
import findspark
findspark.init()

In [ ]:
findspark.find()

'/content/spark-3.1.1-bin-hadoop3.2'

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()